### SparkSession

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('groupby') \
    .getOrCreate()

25/07/02 13:44:25 WARN Utils: Your hostname, khai2612-Latitude-5480 resolves to a loopback address: 127.0.1.1; using 172.16.2.93 instead (on interface wlp2s0)
25/07/02 13:44:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/07/02 13:44:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Reading, creating view, querying (group by) & saving result

#### green

In [ ]:
df_green = spark.read.parquet('data/parquet/green/*/['hour', 'zone']*')

In [4]:
df_green.createOrReplaceTempView('green')

In [5]:
df_green_revenue = spark.sql("""
    SELECT 
        date_trunc('hour', lpep_pickup_datetime) AS hour,
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(*) AS number_records 
    FROM 
        green
    WHERE
        lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY
        1, 2
    """)

In [6]:
df_green_revenue.show(5)

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-17 17:00:00|  41| 638.2700000000001|            49|
|2020-01-02 16:00:00|  66|             229.4|            12|
|2020-01-17 03:00:00|  65|501.28000000000014|            32|
|2020-01-07 19:00:00|  66|             179.5|             9|
|2020-01-13 03:00:00| 247|36.900000000000006|             3|
+-------------------+----+------------------+--------------+
only showing top 5 rows



In [7]:
df_green_revenue.repartition(20).write.parquet('data/report/revenue/green', mode='overwrite')

#### yellow

In [9]:
df_yellow = spark.read.parquet('data/parquet/yellow/*/*')

In [10]:
df_yellow.createOrReplaceTempView('yellow')

In [11]:
df_yellow_revenue = spark.sql("""
    SELECT 
        date_trunc('hour', tpep_pickup_datetime) AS hour,
        PULocationID AS zone,
        SUM(total_amount) AS amount,
        COUNT(1) AS number_records 
    FROM 
        yellow
    WHERE
        tpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY
        1, 2
    """)

In [12]:
df_yellow_revenue.repartition(20).write.parquet('data/report/revenue/yellow', mode='overwrite')